In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,12669
2,Huelva,Confirmados PDIA 14 días,1334
3,Huelva,Tasa PDIA 14 días,"261,18811222082337"
4,Huelva,Confirmados PDIA 7 días,836
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,17
629,Municipio de Huelva sin especificar,Total Confirmados,171
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  12669.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3586.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 681 personas en los últimos 7 días 

Un positivo PCR cada 397 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,12669.0,836.0,1334.0,510743.0,163.683105,261.188112,506.0
Huelva-Costa,7289.0,531.0,863.0,288115.0,184.301407,299.533173,307.0
Condado-Campiña,3758.0,175.0,284.0,155057.0,112.861722,183.158451,128.0
Huelva (capital),3586.0,211.0,362.0,143663.0,146.871498,251.978589,113.0
Isla Cristina,683.0,131.0,216.0,21264.0,616.064710,1015.801354,81.0
Sierra de Huelva-Andévalo Central,1452.0,113.0,151.0,67571.0,167.231505,223.468648,58.0
Lepe,1051.0,45.0,69.0,27431.0,164.047975,251.540228,34.0
Moguer,458.0,35.0,68.0,22088.0,158.457081,307.859471,28.0
Palma del Condado (La),604.0,23.0,49.0,10761.0,213.734783,455.348016,22.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,36.0,26.0,27.0,1305.0,1992.337165,2068.965517,22.0
Isla Cristina,683.0,131.0,216.0,21264.0,616.064710,1015.801354,81.0
Almendro (El),24.0,3.0,8.0,826.0,363.196126,968.523002,0.0
Villalba del Alcor,133.0,25.0,29.0,3338.0,748.951468,868.783703,18.0
Cerro de Andévalo (El),64.0,5.0,19.0,2364.0,211.505922,803.722504,4.0
Aroche,72.0,16.0,17.0,3073.0,520.663846,553.205337,3.0
Corteconcepción,21.0,3.0,3.0,548.0,547.445255,547.445255,1.0
Alosno,64.0,16.0,20.0,3923.0,407.851134,509.813918,14.0
Cumbres Mayores,20.0,1.0,8.0,1755.0,56.980057,455.840456,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,20.0,1.0,8.0,1755.0,56.980057,455.840456,0.0,0.125000
Villablanca,71.0,1.0,4.0,2848.0,35.112360,140.449438,1.0,0.250000
Escacena del Campo,102.0,1.0,4.0,2284.0,43.782837,175.131349,0.0,0.250000
Cerro de Andévalo (El),64.0,5.0,19.0,2364.0,211.505922,803.722504,4.0,0.263158
Almendro (El),24.0,3.0,8.0,826.0,363.196126,968.523002,0.0,0.375000
Lucena del Puerto,114.0,3.0,8.0,3371.0,88.994364,237.318303,2.0,0.375000
Palma del Condado (La),604.0,23.0,49.0,10761.0,213.734783,455.348016,22.0,0.469388
Punta Umbría,254.0,8.0,17.0,15242.0,52.486550,111.533919,4.0,0.470588
Palos de la Frontera,280.0,9.0,19.0,11289.0,79.723625,168.305430,7.0,0.473684
